# Coursera - Applied Data Science - Capstone
Week 3

# Part 1

In [3]:
! pip install beautifulsoup4
! pip install lxml
! pip install html5lib

     |████████████████████████████████| 122kB 911kB/s eta 0:00:01


In [13]:
import requests
import pandas as pd
#import lxml.html as lh
from bs4 import BeautifulSoup

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

retrieve data and create BeautifulSoup object

In [26]:
html = requests.get(url).content
soup = BeautifulSoup(html)

In [27]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else: 
        # create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood
        cell['PostalCode'] = row.p.text[:3] 
        #As postal code contains upto 3 characters extract that using tablerow.p.text
        cell['Borough'] = (row.span.text).split('(')[0]
        #use split ,strip and replace functions for getting Borough and Neighborhood information
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        #Append to the list
        table_contents.append(cell)

# Create a dataframe with list
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [29]:
df.shape

(103, 3)

In [35]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# Part 2

In [33]:
! pip install geocoder

     |████████████████████████████████| 102kB 4.0MB/s ta 0:00:01


Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [42]:
#group the dataframe by borough 
df_postcode = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

df_postcode.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [43]:
#join csv file and df_postcode based on PostalCode 
locgeo_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')
toronto_data = df_postcode.join(locgeo_df, on='PostalCode')

In [44]:
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. 

-add enough Markdown cells to explain what you decided to do and to report any observations you make. 

-generate maps to visualize your neighborhoods and how they cluster together. 

In [46]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Use geopy library to get the latitude and longitude values of Toronto

In [47]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods superimposed on top.

In [48]:
# create map of Toronto  using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

In [49]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


Simplify the above map and segment and cluster only the neighborhoods in Downton Toronto

In [52]:
Downton_data = toronto_data[toronto_data['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downton_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [53]:
Downton_data.shape

(17, 5)

In [54]:
address = 'Downtown Toronto, Ontario'

geolocator = Nominatim(user_agent="doto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Toronto  using latitude and longitude values
map_doto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(Downton_data['Latitude'], Downton_data['Longitude'],  Downton_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_doto)  
    
map_doto

Define Foursquare Credentials and Version

In [55]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Your credentails:
CLIENT_ID: XA0SEEZ4UVQYINFH10RJW2D2SBGA1DJQGNPDUGJNITJ4Y5RW
CLIENT_SECRET:JKSWLYNNROIIRUCEGYS2NPS5KMZCCNZLIQZ1QGLMXW4ZDJ50


get the top 100 venues that are in all neighborhoods in Downtown Toronto within a radius of 500 meters.

In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [58]:
doto_venues = getNearbyVenues(names = Downton_data['Neighborhood'], latitudes = Downton_data['Latitude'], longitudes = Downton_data['Longitude'])
doto_venues.groupby('Neighborhood').count()

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
First Canadian Place, Underground city
Christie


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,65,65,65,65,65,65
Christie,15,15,15,15,15,15
Church and Wellesley,73,73,73,73,73,73
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [59]:
print('There are {} uniques categories.'.format(len(doto_venues['Venue Category'].unique())))

There are 203 uniques categories.


Analyse each Neighborhood

In [62]:
# one hot encoding
doto_onehot = pd.get_dummies(doto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
doto_onehot['Neighborhood'] = doto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [doto_onehot.columns[-1]] + list(doto_onehot.columns[:-1])
doto_onehot = doto_onehot[fixed_columns]

doto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
doto_onehot.shape

(1090, 203)

group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [65]:
doto_grouped = doto_onehot.groupby('Neighborhood').mean().reset_index()

function to sort the venues in descending order.

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create the new dataframe and display the top 10 venues for each neighborhood.

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = doto_grouped['Neighborhood']

for ind in np.arange(doto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(doto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Pharmacy,Seafood Restaurant,Beer Bar,Restaurant,Farmers Market,Liquor Store
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Airport Service,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Coffee Shop,Airport Gate,Airport Lounge
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Japanese Restaurant,Salad Place,Bubble Tea Shop,Thai Restaurant,Department Store
3,Christie,Grocery Store,Café,Park,Baby Store,Coffee Shop,Nightclub,Restaurant,Candy Store,Italian Restaurant,Greek Restaurant
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Mediterranean Restaurant,Café,Pub,Hotel


Run _k_-means to cluster the neighborhood into 5 clusters.

In [80]:
# set number of clusters
kclusters = 5

doto_grouped_clustering = doto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(doto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 4, 0, 0, 0, 0, 0, 0], dtype=int32)

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [81]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

doto_merged = Downton_data

# merge doto_grouped with Downton_data to add latitude/longitude for each neighborhood
doto_merged = doto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

doto_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

Visualize clusters

In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(doto_merged['Latitude'], doto_merged['Longitude'], doto_merged['Neighborhood'], doto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

There is only one large cluster and 4 single coloumn clusters. 

Cluster 1

In [74]:
doto_merged.loc[doto_merged['Cluster Labels'] == 0, doto_merged.columns[[1] + list(range(5, doto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Café,Coffee Shop,Pizza Place,Italian Restaurant,Restaurant,Bakery,Pub,Playground,Japanese Restaurant,Jewelry Store
2,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Mediterranean Restaurant,Café,Pub,Hotel
3,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Theater,Breakfast Spot,Café,Greek Restaurant,Gym / Fitness Center,Health Food Store
4,Downtown Toronto,0,Clothing Store,Coffee Shop,Bubble Tea Shop,Middle Eastern Restaurant,Cosmetics Shop,Café,Japanese Restaurant,Hotel,Bookstore,Italian Restaurant
5,Downtown Toronto,0,Café,Coffee Shop,Gastropub,Cocktail Bar,Cosmetics Shop,Lingerie Store,Seafood Restaurant,Hotel,Moroccan Restaurant,Farmers Market
6,Downtown Toronto,0,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Pharmacy,Seafood Restaurant,Beer Bar,Restaurant,Farmers Market,Liquor Store
7,Downtown Toronto,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Burger Joint,Japanese Restaurant,Salad Place,Bubble Tea Shop,Thai Restaurant,Department Store
8,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Thai Restaurant,Clothing Store,Hotel,Burrito Place,Cosmetics Shop
9,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Scenic Lookout,Fried Chicken Joint,Restaurant,Brewery,Italian Restaurant,Sporting Goods Shop
10,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Deli / Bodega


Cluster 2

In [75]:
doto_merged.loc[doto_merged['Cluster Labels'] == 1, doto_merged.columns[[1] + list(range(5, doto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


Cluster 3

In [76]:
doto_merged.loc[doto_merged['Cluster Labels'] == 2, doto_merged.columns[[1] + list(range(5, doto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Airport Terminal,Airport Service,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Coffee Shop,Airport Gate,Airport Lounge
